# Quiz 1

In [78]:
"""
In this problem set you work with another type of infobox data, audit it,
clean it, come up with a data model, insert it into MongoDB and then run some
queries against your database. The set contains data about Arachnid class
animals.

Your task in this exercise is to parse the file, process only the fields that
are listed in the FIELDS dictionary as keys, and return a list of dictionaries
of cleaned values. 

The following things should be done:
- keys of the dictionary changed according to the mapping in FIELDS dictionary
- trim out redundant description in parenthesis from the 'rdf-schema#label'
  field, like "(spider)"
- if 'name' is "NULL" or contains non-alphanumeric characters, set it to the
  same value as 'label'.
- if a value of a field is "NULL", convert it to None
- if there is a value in 'synonym', it should be converted to an array (list)
  by stripping the "{}" characters and splitting the string on "|". Rest of the
  cleanup is up to you, e.g. removing "*" prefixes etc. If there is a singular
  synonym, the value should still be formatted in a list.
- strip leading and ending whitespace from all fields, if there is any
- the output structure should be as follows:

[ { 'label': 'Argiope',
    'uri': 'http://dbpedia.org/resource/Argiope_(spider)',
    'description': 'The genus Argiope includes rather large and spectacular spiders that often ...',
    'name': 'Argiope',
    'synonym': ["One", "Two"],
    'classification': {
                      'family': 'Orb-weaver spider',
                      'class': 'Arachnid',
                      'phylum': 'Arthropod',
                      'order': 'Spider',
                      'kingdom': 'Animal',
                      'genus': None
                      }
  },
  { 'label': ... , }, ...
]

  * Note that the value associated with the classification key is a dictionary
    with taxonomic labels.
"""
import codecs
import csv
import json
import pprint
import re

DATAFILE = 'data/arachnid.csv'
FIELDS ={'rdf-schema#label': 'label',
         'URI': 'uri',
         'rdf-schema#comment': 'description',
         'synonym': 'synonym',
         'name': 'name',
         'family_label': 'family',
         'class_label': 'class',
         'phylum_label': 'phylum',
         'order_label': 'order',
         'kingdom_label': 'kingdom',
         'genus_label': 'genus'}

In [79]:
name1 = 'kjnasdkfj@@opsadf'
name2 = 'dskafjno2832wejonfkjerfo3SDFSD23'

In [80]:
if re.search('[^A-Za-z\d\s:]', name1):
    print('asdfa')

asdfa


In [81]:
if re.search('[^A-Za-z\d\s:]', name2):
    print('asdkfask')

In [82]:
def process_file(filename, fields):

    process_fields = fields.keys()
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(3):
            l = reader.next()

        for line in reader:
            # Rename keys
            record = {new_k: line[old_k] for old_k, new_k in FIELDS.items()}
            
            # Remove redundant information in parenthesis
            record['label'] = re.sub('\(.*\)', '', record['label']).strip()
            
            # Replace bad names
            if (record['name'] == 'NULL') or re.search('[^A-Za-z\d\s:]', record['name']):
                record['name'] = record['label']
            
            # Replace NULL fields
            for field in set(record.keys()) - {'name'}:
                if record[field] == 'NULL':
                    record[field] = None
            
            # Synonym to list
            if record['synonym'] is not None:
                record['synonym'] = map(str.strip, record['synonym'].strip('{}').split('|'))
            
            # Strip whitespaces
            for field in record.keys():
                if isinstance(record[field], str):
                    record[field] = record[field].strip()
            
            # Correct the structure
            classification_fields = [
                'family',
                'class',
                'phylum',
                'order',
                'kingdom',
                'genus'
            ]
            record['classification'] = dict()
            for field in classification_fields:
                record['classification'][field] = record.pop(field)
            
            # Add to the general list
            data.append(record)
    return data

In [84]:
# data = process_file(DATAFILE, FIELDS)
# data

In [85]:
def parse_array(v):
    if (v[0] == "{") and (v[-1] == "}"):
        v = v.lstrip("{")
        v = v.rstrip("}")
        v_array = v.split("|")
        v_array = [i.strip() for i in v_array]
        return v_array
    return [v]


def test():
    data = process_file(DATAFILE, FIELDS)
    print "Your first entry:"
    pprint.pprint(data[0])
    first_entry = {
        "synonym": None, 
        "name": "Argiope", 
        "classification": {
            "kingdom": "Animal", 
            "family": "Orb-weaver spider", 
            "order": "Spider", 
            "phylum": "Arthropod", 
            "genus": None, 
            "class": "Arachnid"
        }, 
        "uri": "http://dbpedia.org/resource/Argiope_(spider)", 
        "label": "Argiope", 
        "description": "The genus Argiope includes rather large and spectacular spiders that often have a strikingly coloured abdomen. These spiders are distributed throughout the world. Most countries in tropical or temperate climates host one or more species that are similar in appearance. The etymology of the name is from a Greek name meaning silver-faced."
    }

    assert len(data) == 76
    assert data[0] == first_entry
    assert data[17]["name"] == "Ogdenia"
    assert data[48]["label"] == "Hydrachnidiae"
    assert data[14]["synonym"] == ["Cyrene Peckham & Peckham"]

test()

Your first entry:
{'classification': {'class': 'Arachnid',
                    'family': 'Orb-weaver spider',
                    'genus': None,
                    'kingdom': 'Animal',
                    'order': 'Spider',
                    'phylum': 'Arthropod'},
 'description': 'The genus Argiope includes rather large and spectacular spiders that often have a strikingly coloured abdomen. These spiders are distributed throughout the world. Most countries in tropical or temperate climates host one or more species that are similar in appearance. The etymology of the name is from a Greek name meaning silver-faced.',
 'label': 'Argiope',
 'name': 'Argiope',
 'synonym': None,
 'uri': 'http://dbpedia.org/resource/Argiope_(spider)'}


# Quiz 2

In [89]:
"""
Complete the insert_data function to insert the data into MongoDB.
"""

import json

def insert_data(data, db):
    db.arachnid.insert_many(data)

In [90]:
if True:
    
    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017")
    db = client.examples

    with open('data/arachnid.json') as f:
        data = json.loads(f.read())
        insert_data(data, db)
        print db.arachnid.find_one()

{u'synonym': None, u'description': u'The genus Argiope includes rather large and spectacular spiders that often have a strikingly coloured abdomen. These spiders are distributed throughout the world. Most countries in tropical or temperate climates host one or more species that are similar in appearance. The etymology of the name is from a Greek name meaning silver-faced.', u'classification': {u'kingdom': u'Animal', u'family': u'Orb-weaver spider', u'order': u'Spider', u'phylum': u'Arthropod', u'genus': None, u'class': u'Arachnid'}, u'uri': u'http://dbpedia.org/resource/Argiope_(spider)', u'label': u'Argiope', u'_id': ObjectId('5ca655f7873d81058768e2e5'), u'name': u'Argiope'}
